<a href="https://colab.research.google.com/github/shirishagugulothu/GEN_AI_ASSIGNMENT_6/blob/main/Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install necessary libraries
!pip install transformers chromadb pymupdf sentence-transformers

In [ ]:
# 2. Upload PDF via Colab
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# 3. Extract text from PDF
import fitz
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

document_text = extract_text_from_pdf(pdf_path)

# 4. Chunk the text
import textwrap
def chunk_text(text, chunk_size=300):
    return textwrap.wrap(text, width=chunk_size)

chunks = chunk_text(document_text)
print(f"Number of chunks: {len(chunks)}")
print("sample_chunk:", chunks[0])

# 5. Load embedding model
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# 6. Compute embeddings for chunks
embeddings = embed_model.encode(chunks)

# 7. Initialize ChromaDB and create collection
import chromadb

# Option A: For persistent disk storage:
client = chromadb.PersistentClient(path="chroma_db")  # data saved in the "chroma_db" directory

# Option B: (Alternative) Use in-memory client (no persistence needed):
# client = chromadb.Client()

collection = client.get_or_create_collection(name="documents")

# 8. Add documents + embeddings into collection
collection.add(
    documents=chunks,
    embeddings=embeddings.tolist(),
    metadatas=[{"chunk_id": i} for i in range(len(chunks))],
    ids=[str(i) for i in range(len(chunks))]
)

# 9. Set up text generation
from transformers import pipeline
generator = pipeline("text2text-generation", model="google/flan-t5-small")

# 10. Define retrieval + generation function
def retrieve_and_answer(query, top_k=1):
    results = collection.query(
        query_texts=[query],
        n_results=top_k
    )
    retrieved = results['documents'][0]
    context = " ".join(retrieved)
    prompt = f"context: {context} \n\nQuestion: {query}\nAnswer:"
    result = generator(prompt, max_length=80)
    return result[0]['generated_text']

# Example usage
# Question = "The linux kernel organises I/O operations into how many distinct layers? and what are they?"
# Answer = retrieve_and_answer(Question)
# print("Q:", Question)
# print("A:", Answer)


Saving Module 5.pdf to Module 5 (2).pdf
Number of chunks: 39
sample_chunk: A Guide to Linux I/O and Disk Management  Chapter 1: The Linux I/O Subsystem  Introduction  Input/Output (I/O) is the fundamental process of communication between a  computer's central processing unit (CPU) and the outside world, including storage  devices, networks, and user peripherals. In Linux,


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:08<00:00, 10.2MiB/s]
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Q: The linux kernel organises I/O operations into how many distinct layers? and what are they?
A: four


In [ ]:
# Example usage
Question = "What is the fundamental process of communication between a computer's central processing unit (CPU) and the outside world?"
Answer = retrieve_and_answer(Question)
print("Q:", Question)
print("A:", Answer)

Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Q: What is the fundamental process of communication between a computer's central processing unit (CPU) and the outside world?
A: Input/Output
